In [80]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

from lobio.lob.limit_order import LimitOrder
from lobio.lob.order_book import OrderBook
from lobio.lob.price_level import PriceLevel, TraderId, Side

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [81]:
price_level = PriceLevel(LimitOrder(10, 1, Side.BUY, TraderId.MARKET))

price_level.add_limit_order(LimitOrder(10, 0.5, Side.BUY, TraderId.MARKET))

assert price_level == PriceLevel(LimitOrder(10, 1.5, Side.BUY, TraderId.MARKET))

price_level.add_limit_order(LimitOrder(10, 0.2, Side.BUY, TraderId.MM))

assert price_level.traders_order == [LimitOrder(10, 1.5, Side.BUY, TraderId.MARKET), LimitOrder(10, 0.2, Side.BUY, TraderId.MM)]
assert price_level.quote == 1.7

price_level.add_limit_order(LimitOrder(10, 0.15, Side.BUY, TraderId.MARKET))

assert price_level.traders_order == [LimitOrder(10, 1.5, Side.BUY, TraderId.MARKET), 
                                    LimitOrder(10, 0.2, Side.BUY, TraderId.MM), 
                                    LimitOrder(10, 0.15, Side.BUY, TraderId.MARKET)]
assert price_level.quote == 1.85

In [82]:
price_level = PriceLevel(LimitOrder(20, 0.5, Side.BUY, TraderId.MARKET))

remain_quote, data = price_level.execute_limit_order(0.2)

assert (remain_quote == 0.0)
assert (data == {TraderId.MARKET: 0.2})
assert (price_level == PriceLevel(LimitOrder(20, 0.3, Side.BUY, TraderId.MARKET)))

price_level.add_limit_order(LimitOrder(20, 0.05, Side.BUY, TraderId.MM))
remain_quote, data = price_level.execute_limit_order(0.31)

assert (remain_quote == 0.0)
assert (data == {TraderId.MARKET: 0.3, TraderId.MM: 0.01})
assert (price_level == PriceLevel(LimitOrder(20, 0.04, Side.BUY, TraderId.MM)))

remain_quote, data = price_level.execute_limit_order(1)

assert (remain_quote == 0.96)
assert (data == {TraderId.MM: 0.04})
assert (price_level.quote == 0 and price_level.traders_order == [])

In [83]:
init_lob = {"lastUpdateId": 1, "bids": [[90, 1]], "asks": [[100, 1]]}

diffs = [[2, [[90, 1]], [[100, 1]]], 
         [3, [[89.99, 0.99]], [[100.01, 3]]], 
         [4, [[94.99, 0.0001], [89.99, -0.01]], [[95, 0.0001], [100.01, -1]]],
         [5, [], [[95.01, 0.3], [95.5, 1], [100, 0.1]]],
         [6, [[87, 0.01], [90, 0.01], [94.99, 1]], []],
         [7, [[90, -1.5], [87, -10]], [[95.01, -0.29], [100, -2.1]]],
         [8, [[89.99, -0.98]], [[95, -0.0001]]],
         [9, [[90.01, 0.51]], [[95, 1]]]]

In [84]:
ob = OrderBook.create_lob_init(init_lob)

ob.apply_historical_update(diffs[0])
assert ob == OrderBook.create_lob_init({"lastUpdateId": 2, 
                                       "bids": [[90, 2]], 
                                       "asks": [[100, 2]]})

ob.apply_historical_update(diffs[1])
assert ob == OrderBook.create_lob_init({"lastUpdateId": 3, 
                                       "bids": [[90, 2], [89.99, 0.99]], 
                                       "asks": [[100, 2], [100.01, 3]]})

ob.apply_historical_update(diffs[2])
assert ob == OrderBook.create_lob_init({"lastUpdateId": 4, 
                                       "bids": [[94.99, 0.0001], [90, 2], [89.99, 0.98]], 
                                       "asks": [[95, 0.0001], [100, 2], [100.01, 2]]})

ob.apply_historical_update(diffs[3])
assert ob == OrderBook.create_lob_init({"lastUpdateId": 5, 
                                       "bids": [[94.99, 0.0001], [90, 2], [89.99, 0.98]], 
                                        "asks": [[95, 0.0001], [95.01, 0.3], [95.5, 1], [100, 2.1], [100.01, 2]]})

ob.apply_historical_update(diffs[4])
assert ob == OrderBook.create_lob_init({"lastUpdateId": 6, 
                                       "bids": [[94.99, 1.0001], [90, 2.01], [89.99, 0.98], [87, 0.01]], 
                                       "asks": [[95, 0.0001], [95.01, 0.3], [95.5, 1], [100, 2.1], [100.01, 2]]})

ob.apply_historical_update(diffs[5])
assert ob == OrderBook.create_lob_init({"lastUpdateId": 7, 
                                       "bids": [[94.99, 1.0001], [90, 0.51], [89.99, 0.98]],
                                       "asks": [[95, 0.0001], [95.01, 0.01], [95.5, 1], [100.01, 2]]})

ob.apply_historical_update(diffs[6])
assert ob == OrderBook.create_lob_init({"lastUpdateId": 8, 
                                       "bids": [[94.99, 1.0001], [90, 0.51]], 
                                       "asks": [[95.01, 0.01], [95.5, 1], [100.01, 2]]})

ob.apply_historical_update(diffs[7])
assert ob == OrderBook.create_lob_init({"lastUpdateId": 9, 
                                       "bids": [[94.99, 1.0001], [90.01, 0.51], [90, 0.51]],
                                       "asks": [[95, 1], [95.01, 0.01], [95.5, 1], [100.01, 2]]})

In [85]:
init_lob = {"lastUpdateId": 1, "bids": [[48.37, 0.3], [48, 0.2]], "asks": [[50.45, 0.5], [50.47, 0.5]]}

In [86]:
ob = OrderBook.create_lob_init(init_lob)

data = ob.set_limit_order(LimitOrder(48.38, 0.1, Side.SELL, TraderId.MARKET))
assert ob == OrderBook.create_lob_init({"lastUpdateId": 2,
                                        "bids": [[48.37, 0.3], [48, 0.2]],
                                        "asks": [[48.38, 0.1], [50.45, 0.5], [50.47, 0.5]]})
assert data == {}

data = ob.set_limit_order(LimitOrder(48.38, 0.101, Side.BUY, TraderId.MARKET))
assert ob == OrderBook.create_lob_init({"lastUpdateId": 3,
                                        "bids": [[48.38, 0.001], [48.37, 0.3], [48, 0.2]],
                                        "asks": [[50.45, 0.5], [50.47, 0.5]]})
assert data == {TraderId.MARKET: [0.1, 0.1 * 48.38]}

data = ob.set_limit_order(LimitOrder(48.38, 0.001, Side.SELL, TraderId.MARKET))
assert ob == OrderBook.create_lob_init({"lastUpdateId": 4,
                                        "bids": [[48.37, 0.3], [48, 0.2]],
                                        "asks": [[50.45, 0.5], [50.47, 0.5]]})
assert data == {TraderId.MARKET: [0.001, 0.001 * 48.38]}

data = ob.set_limit_order(LimitOrder(50.45, 0.5, Side.BUY, TraderId.MARKET))
assert ob == OrderBook.create_lob_init({"lastUpdateId": 5,
                                        "bids": [[48.37, 0.3], [48, 0.2]],
                                        "asks": [[50.47, 0.5]]})
assert data == {TraderId.MARKET: [0.5, 0.5 * 50.45]}

data = ob.set_limit_order(LimitOrder(48, 0.49, Side.SELL, TraderId.MARKET))
assert ob == OrderBook.create_lob_init({"lastUpdateId": 6,
                                        "bids": [[48, 0.01]],
                                        "asks": [[50.47, 0.5]]})
assert data == {TraderId.MARKET: [0.49, 0.3 * 48.37 + 0.19 * 48]}

data = ob.set_limit_order(LimitOrder(50.47, 0.5, Side.BUY, TraderId.MARKET))
assert ob == OrderBook.create_lob_init({"lastUpdateId": 7,
                                        "bids": [[48, 0.01]],
                                        "asks": []})
assert data == {TraderId.MARKET: [0.5, 0.5 * 50.47]}

In [87]:
init_lob = {"lastUpdateId": 1, "bids": [], "asks": []}

In [115]:
ob = OrderBook.create_lob_init(init_lob)

data = ob.set_limit_order(LimitOrder(33, 0.5, Side.SELL, TraderId.MARKET))
assert ob == OrderBook.create_lob_init({"lastUpdateId": 2,
                                        "bids": [],
                                        "asks": [[33, 0.5]]})
assert data == {}

data = ob.set_limit_order(LimitOrder(34, 0.3, Side.BUY, TraderId.MM))
assert ob == OrderBook.create_lob_init({"lastUpdateId": 3,
                                        "bids": [],
                                        "asks": [[33, 0.2]]})
assert data == {TraderId.MARKET: [0.3, 0.3 * 33]}

data = ob.set_limit_order(LimitOrder(32.5, 0.4, Side.BUY, TraderId.MM))
ob_true = OrderBook.create_lob_init({"lastUpdateId": 4,
                                        "bids": [[32.5, 0.4]],
                                        "asks": [[33, 0.2]]})

ob_true.bids[0].traders_order[0].trader_id = TraderId.MM
assert ob == ob_true
assert data == {}

data = ob.set_limit_order(LimitOrder(32.5, 0.2, Side.SELL, TraderId.MARKET))
ob_true = OrderBook.create_lob_init({"lastUpdateId": 5,
                                        "bids": [[32.5, 0.2]],
                                        "asks": [[33, 0.2]]})
ob_true.bids[0].traders_order[0].trader_id = TraderId.MM
assert ob == ob_true
assert data == {TraderId.MM: [0.2, 0.2 * 32.5]}

data = ob.set_limit_order(LimitOrder(32.7, 1, Side.BUY, TraderId.MARKET))
ob_true = OrderBook.create_lob_init({"lastUpdateId": 6,
                                        "bids": [[32.7, 1], [32.5, 0.2]],
                                        "asks": [[33, 0.2]]})
ob_true.bids[1].traders_order[0].trader_id = TraderId.MM
assert ob == ob_true
assert data == {}

data = ob.set_limit_order(LimitOrder(33, 0.7, Side.BUY, TraderId.MM))
ob_true = OrderBook.create_lob_init({"lastUpdateId": 7,
                                        "bids": [[33, 0.5], [32.7, 1], [32.5, 0.2]],
                                        "asks": []})
ob_true.bids[0].traders_order[0].trader_id = TraderId.MM
ob_true.bids[2].traders_order[0].trader_id = TraderId.MM
assert ob == ob_true
assert data == {TraderId.MARKET: [0.2, 0.2 * 33]}